# Participation of a wind power producer in day-ahead electricity markets
----------------
**This material was developed by:**

Dr. Nikolaos Paterakis (market participation algorithm) | n.paterakis@tue.nl

Dr. Juan Giraldo Chavarriaga (the EMGA platform) | j.s.giraldo.chavarriaga@tue.nl

**Version:** Mar. 29, 2020

----------------
This notebook provides a simple, yet quite comprehensive, pipeline for preparing bids for a price-taker wind producer participating 
in the day-ahead electricity market by considering the effects of the imbalance mechanism. 

The stochastic programming model is based on the following book (which you can access by using TU/e VPN):
> A. J. Conejo, M. Carrión, J. M. Morales, [*Decision making under uncertainty in electricity markets*](https://link.springer.com/book/10.1007/978-1-4419-7421-1), Springer US, 2010.

In the original model day-ahead market prices, imbalance price ratios and available wind energy are all considered to be uncertain variables. For simplicity, in this case we are considering that day-ahead market prices are perfectly known and that imbalance price ratios are given by a fixed set of scenarios for every trading day. Your task is to generate a scenario set for available wind energy for the next trading day. 

Also, without loss of generality and in order to simplify data handling, we consider that the day-ahead market is cleared excactly at the beggining of the day of interest (midnight). In reality, the decisions have to be made several hours before the beggining of the day of interest as it was discussed during the lecture.

**What is recommended to change?**

Technically, you can modify any parts of the code. Nevertheless, it is recommended that you experiment with the following parts of the pipeline:
1. Use different regression models that are readily available in [scikit-learn](https://scikit-learn.org/stable/), feature selection and data processing techniques. As it was mentioned during the introductory lecture, the linear regression model is not necessarily the most suitable one for predicting/generating scenarios for available wind energy. Depending on how fast your computer is, you might as well experiment with other machine learning models (e.g., [Keras](https://keras.io/) is a popular neural network module -- in that case, the [tensorflow](https://www.tensorflow.org/) backend is strongly recommended). 
2. Modify the risk-aversion factor $\beta$. 
3. Change the number of scenarios generated for the available wind energy.
4. You can generate a set of non-equiprobable scenarios by reducing the initially generated set of scenarios in terms of, e.g., a [clustering approach](https://scikit-learn.org/stable/modules/clustering.html).
5. You can reduce the number of scenarios that are provided for the imbalance prices.

**Instructions to prepare a bid for each trading day:**
1. Download the most recent data from the game platform (https://sa-emga.herokuapp.com/)
2. Update the forecasts/scenario set 
3. Execute the optimization model 
4. Upload the .csv file containing the bid quantities for each time period on the platform 

<img src="img/FLOWCHART.png" alt="Drawing" style="width: 350px;"/>

**Known caveats:**
1. The variable $periodsPast$ triggers an error if reduced to less than 144. There is no problem with reducing the number of features by means of a feature selection technique.

**Make sure that you have installed the following libraries:**
- numpy 
- pandas
- xlrd
- openpyxl
- scipy
- statsmodels
- matplotlib
- scikit-learn
- pyomo

Normally, these libraries will install all the necessary dependencies. Nonetheless, be alert for errors that indicate that specific packages might be missing and install them.

**Also, it is assumed that you have installed the [Gurobi](https://www.gurobi.com/downloads/end-user-license-agreement-academic/) solver.**


**<span style="color:red">It is recommended to RESTART \& CLEAR OUTPUT via Kernel in the menu above everytime you make changes in the scripts or you want to repeat the process of generating offers.</span>**

-----------------------------------------

## Import packages

In [ ]:
import numpy, pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.metrics import mean_squared_error
from scripts.forecastingUtils.foreUtils_2020 import *
from scripts.forecastingUtils.foreDisplays_2020 import *
from scripts.generalUtils_2020 import *
from scripts.optimizationUtils.stochasticProgrammingModel import *
from scripts.optimizationUtils.reportingUtils import *
from scripts.optimizationUtils.plotUtils import *

numpy.random.seed(10)

## Available wind energy scenario generation

### Basic settings

- Download the file containing the historical wind speed data from the platform (**windSpeed_YYYY-MM-DD.csv**).
- Save the file in the "data" directory.
- Always make sure to change the variable **inputFileName** to the filename of interest and the **firstDateTest** to the beggining of the date of interest.

In [ ]:
inputFileName = 'windSpeed_2020-03-30.csv' #latest historical data
firstDateTest = '2020-03-30 00:00:00'#date for which scenarios are going to be generated (the last date in the historical data file)

turbineRatedPower = 2500 # in kW -- DO NOT modify
windfarmRatedPower = 25000 #in kW -- DO NOT modify
periodsFuture= 144 #a full day (wind speed is given in 10-minute intervals)
periodsPast= 144*3 #how many past periods are used to predict the next period (before feature selection)
daysHistory= 7 #how many days are used in the training set
numScenarios = 10 #number of scenarios

firstDateTrain = pandas.to_datetime(firstDateTest)-pandas.Timedelta(str(daysHistory)+'D')

featureSelection = True
plotResidualDiagnostics = False
scenarios = numpy.zeros((numScenarios, periodsFuture)) #initialize matrix of scenarios

inputDataDir = 'data/'
outputDataDir = 'data/'
outputDataDir1 = firstDateTest.split(' ')[0]
outputFilename = 'wind_'+outputDataDir1+'.csv'

### Data preparation

Data pre-processing (e.g., normalization) should be based on data from the training set. However, in this basic script, the day to be predicted is included in the input file with all its entries as zeros and the whole series is used in order to estimate a standard scaler model. Empirically, it shouldn't affect much the process in this particular setup, but this is something to consider if you are willing to further modify the pipeline (e.g., to perform better model selection or apply cross-validation).

In [ ]:
# Load and scale data
wspeed = pandas.read_csv(inputDataDir+inputFileName, index_col=0, parse_dates=True, dayfirst=True)['speed']

scaler = StandardScaler()
wspeed.loc[:] = scaler.fit_transform(wspeed.values.reshape(-1,1))[:,0]

# Create datasets
C = createDataSet(wspeed, periodsPast)
trainSet, testSet = splitTrainTest(C, firstDateTrain=firstDateTrain, firstDateTest=firstDateTest, value=10, unit='min')
trainX, trainY = splitXY(trainSet)
testX, testY = splitXY(testSet)
print('Train X: ', trainX.shape, 'Train Y: ', trainY.shape,'Test X: ', testX.shape,'Test Y: ', testY.shape)

# Feature selection
if featureSelection:
    print('Starting feature selection!')
    mask = feature_selection(trainX, trainY, 'rfe')
    trainX = trainX[:,mask]
    print('Done feature selection! New feature matrix size: ', trainX.shape)

else:
    mask = None
    print('No feature selection is applied!')

### Prediction model

The next block genereted the prediction model. If you want to experiement with other prediction models, go to **'scripts/forecastingUtils/foreUtils_2020.py'** and modify the **createPredictionModel** method. 

In [ ]:
# Generate prediction model
model, res, stdevRes = createPredictionModel(trainX, trainY, method='LR')
print('Residual mean: ', numpy.mean(res), 'Residual stdev: ', stdevRes)

# Plot diagnostics on residuals
if plotResidualDiagnostics:
    plot_fit(model.predict(trainX), trainY)
    plot_res_autocor(res)
    plot_res_hist(res)
else:
    print('No residual diagnostics are plotted!')

# Generate scenarios
for j in range(numScenarios):
    arrayActual, arrayPredicted = forecastForward(testSet, testX, model, scaler, periodsFuture, stdevRes, mask=mask, testY=testY, positivityRequirement=True)
    arrayActual, arrayPredicted = scaler.inverse_transform(arrayActual), scaler.inverse_transform(arrayPredicted)
    scenarios[j, :] = arrayPredicted

### Visualizations and reporting

The following block:
- visualizes the outcome of wind energy scenascenario generation
- generates the "scenario tree" (a number of files in the -automatically created- 'data/YYY-MM-DD' directory)

Make sure that the file **'ratioScenarios_2020.csv'** is in the 'data' directory.

In [ ]:
plot_windSpeedScenarios(scenarios, arrayActual)
scenariosPower = powerG126(scenarios)
actualPower = powerG126(arrayActual.reshape(1,-1))
plot_windPowerScenarios(scenariosPower, actualPower)

outputDir = createDataDirectory(outputDataDir, outputDataDir1)
scenariosPowerDf = pandas.DataFrame(data=(scenariosPower*(windfarmRatedPower/turbineRatedPower))/1000, index=['s'+str(s) for s in range(1, numScenarios+1)], columns=['t'+str(t) for t in range(1, 25)])
scenariosPowerDf.to_csv(outputDir+outputFilename)
generateScenarioTree(outputDataDir, outputDataDir1)

## Stochastic programming model 


**<span style="color:red">ATTENTION!</span>** Before running the following blocks make sure that you have placed the file containing the day-ahead prices (**'DAP_YYY-MM-DD.csv'**) in the directory that was created in the previous step (the name of the directory is the same as the date for which you are bidding).

**<span style="color:red">DO NOT! modify the following block!</span>** 

(you still have to run it)

---------------------------------------

In [ ]:
folderName = outputDataDir1

#--- Define basic I/O data
fileDAP = 'data/'+str(folderName)+'/DAP_'+str(folderName)+'.csv'
fileImNeg = 'data/'+str(folderName)+'/tree_imNeg_'+str(folderName)+'.csv'
fileImPos = 'data/'+str(folderName)+'/tree_imPos_'+str(folderName)+'.csv'
fileWind = 'data/'+str(folderName)+'/tree_wind_'+str(folderName)+'.csv'
fileProbs = 'data/'+str(folderName)+'/tree_probs_'+str(folderName)+'.csv'

outDir = 'data/'+str(folderName)+'/'
reportFileName = outDir+'report_'+str(folderName)+'.xlsx'
bidFileName = outDir+'bid_'+str(folderName)+'.csv'

#--- Load data
daP = pandas.read_csv(fileDAP, index_col=0)
wind = pandas.read_csv(fileWind, index_col=0)
rPlus = pandas.read_csv(fileImPos, index_col=0)
rMinus = pandas.read_csv(fileImNeg, index_col=0)
probs = pandas.read_csv(fileProbs, index_col=0)

---------------------

### Market participation algorithm
The following block executes the market participation algorithm. In the directory **YYYY-MM-DD** two files are generated:
- bid_YYY-MM-DD.csv (contains the offer)
- report_2020-02-09.xlsx (solution details for further inspection)

In [ ]:
#--- Execute optimization model
alpha = 0.9
beta = 0.9
a, b = stochasticRisk(daP, wind, rPlus, rMinus, probs, alpha, beta)
 
#--- Report and visualize
displayReport(b)
saveReport(b, reportFileName, bidFileName)

plot_bid(b[1])
plot_profit_distribution(b[2], b[0])
plot_hourly_imbalance_dists(b[3], b[4], b[5])